In [3]:
import numpy as np
# import jax.numpy as jnp
import random
import math
import random
import json

In [2]:
import json

def addToJson(crossover_type, mutation_type, tuple1,fitness):

  # Get two strings and a tuple
  string1 = crossover_type
  string2 = mutation_type
  tuple1 = tuple1

  # Load the existing JSON data (if it exists)
  data = {}
  try:
    with open('data.json') as f:
      existing_data = json.load(f)
      data.update(existing_data)
  except FileNotFoundError:
    pass

  # Find the next available index based on existing keys
  next_index = 0
  if string1 + "-" + string2 in data : 
    while str(next_index) in data[string1 + "-" + string2]:
      next_index += 1
  else :
    data[string1 + "-" + string2] = {}

  print(next_index)
  # Add the tuple to the JSON data
  data[string1 + "-" + string2][str(next_index)] = {
    "ARRAY": tuple1,
    "FINTESS": fitness
  }

  # Write the updated JSON data to the file
  with open('data.json', 'w') as f:
    json.dump(data, f, indent=2)


In [163]:
addToJson('pmx', 'inversion' ,([61, 51, 87, 25, 68, 69, 73, 50, 39, 33, 76, 13, 95, 82, 40, 54, 67,
       93, 28, 89, 56,  8,  1, 98, 23, 77, 60, 35, 86, 57,  9,  7, 34, 85,
       64,  2, 44, 96, 78, 37,  5, 30, 48, 52,  0, 41, 71, 14, 29, 46,  3,
       43, 55, 83, 27, 12, 20, 62, 45, 32, 91, 15, 11, 17, 74, 72, 21, 59,
       99, 36, 63, 10, 84, 24, 38, 18, 79, 90, 16, 70, 94, 22, 88, 53, 49,
        6, 75, 19, 66, 26, 65,  4, 97, 42, 80, 31, 92, 47, 58, 81]), 33597)

29


In [1]:
# def distance(p1, p2):
#   """
#   محاسبه فاصله اقلیدسی دو نقطه

#   Args:
#     p1: نقطه اول
#     p2: نقطه دوم

#   Returns:
#     فاصله اقلیدسی بین دو نقطه
#   """

#   x1, y1 = p1
#   x2, y2 = p2

#   dx = x1 - x2
#   dy = y1 - y2

#   return math.sqrt(dx**2 + dy**2)

In [2]:
# def calculate_distance(cities, solution):
#     total_distance = 0
#     for i in range(len(solution) - 1):
#         city1 = cities[solution[i]]
#         city2 = cities[solution[i + 1]]
#         distance = ((city1[1] - city2[1])**2 + (city1[2] - city2[2])**2)**0.5
#         total_distance += distance
#     return total_distance

In [2]:
# import numpy as np
# def randomGeneration():
#   return np.random.permutation(100)
# import jax.numpy as jnp
from jax import random

def randomGeneration(key):
  return random.permutation(key, 100)

ModuleNotFoundError: jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.

In [4]:
def read_city_data(filename):
    city_data = []
    with open(filename, 'r') as f:
        for line in f:
            city, lat, lon = line.strip().split(' ')
            city_data.append([city, int(lat), int(lon)])
            # check city is good verification
            # id , lat (x), long(y) in array
    return city_data

In [8]:
city_data = read_city_data('sample_data/city-cor.txt')
print(len(city_data))
# for city, lat, lon in city_data:
#     print(f"City: {city}, Latitude: {lat}, Longitude: {lon}")

100


In [9]:
def read_city_answer(filename):
    city_answer = []
    with open(filename, 'r') as f:
        for line in f:
            city_index = int(line)
            city_answer.append(city_index)
            # id , lat (x), long(y) in tupple
    return city_answer

In [10]:
city_ans = read_city_answer('sample_data/answer.txt')
# city_ans

In [11]:
print(city_data[0])

['1', 1380, 939]


In [12]:
def generate_random_chromosomes(population_size):
    chromosomes = []
    for _ in range(population_size):
        chromosome = randomGeneration()
        # random.shuffle(chromosome)
        chromosomes.append(chromosome)
    return chromosomes

In [13]:
def calculate_fitness(chromosome, cities=city_data):
    total_distance = 0
    if len(chromosome) != 100 :
      raise ValueError
    for i in range(100):
        city1_index = chromosome[i-1]
        city2_index = chromosome[i]

        city1 = cities[city1_index-1]
        city2 = cities[city2_index-1]

        latitude_diff = city1[1] - city2[1]
        longitude_diff = city1[2] - city2[2]

        distance = math.sqrt(latitude_diff**2 + longitude_diff**2)
        total_distance += distance
    return int(total_distance // 1)

In [16]:
calculate_fitness(city_ans)

21285

In [17]:
# def calculate_fitness_np(chromosome, cities=city_data):
#     if len(chromosome) != 100 :
#         print('wrong chromosome length')
#         raise ValueError

#     total_distance = 0
#     for i in range(100):
#         city1_index = chromosome[i-1]
#         city2_index = chromosome[i]

#         city1 = np.array(cities[city1_index-1][1:])
#         city2 = np.array(cities[city2_index-1][1:])

#         distance = np.linalg.norm(city1 - city2)
#         total_distance += distance

#     return int(total_distance // 1)
import jax.numpy as jnp
from jax import lax

def calculate_fitness_np(chromosome, cities=city_data):
    if len(chromosome) != 100 :
        print('wrong chromosome length')
        raise ValueError

    def body_fun(i, total_distance):
        city1_index = chromosome[i-1]
        city2_index = chromosome[i]

        city1 = jnp.array(cities[city1_index-1][1:])
        city2 = jnp.array(cities[city2_index-1][1:])

        distance = jnp.linalg.norm(city1 - city2)
        total_distance += distance

        return total_distance

    total_distance = lax.fori_loop(0, 100, body_fun, 0)

    return int(total_distance // 1)

In [18]:
calculate_fitness_np(city_ans)

21285

In [19]:
population = generate_random_chromosomes(500)
print(population[0])

[46 57 86 59 83 67 97  0  3  9 80 68 60 38 25  8 22 42 15 88 70 79 39 51
 16 56 28 19 12 77 69 82 92 24 13 48 50 49 98 44  7 91 37  5 47 14 53 71
 36 41 29 54 32 65 20 66 11 40 27 78 87  6 73 23 33 10 72  4 94 93 75 74
 55 26  1 76 96 31  2 90 18 61 30 17 52 81 62 64 63 58 35 95 84 45 21 34
 85 89 99 43]


In [20]:
# for chro in population :
#   fitforchro = calculate_fitness(chro)
#   if fitforchro < 21285:
#     print(fitforchro)

In [ ]:
# fitness_values = []
# for chromosome in population :
#     fitness_value = calculate_fitness(chromosome, cities = city_data)
#     fitness_values.append(fitness_value)
# fitness_values

In [ ]:
fitness_values = np.array([calculate_fitness(chromosome, cities=city_data) for chromosome in population])
print(fitness_values)

[164163 179856 170598 158845 185933 172576 173113 174694 178062 165858
 174669 162330 175809 161074 178904 155866 173669 156670 157704 180331
 180151 182149 176601 175153 185082 172524 183485 167576 164989 170356
 177925 158429 175852 168069 176584 181132 185986 176002 165545 164294
 163630 172452 181466 183517 173991 161733 173796 174563 176816 180787
 170612 170075 162641 180618 170411 178927 174909 178947 178940 167947
 158395 154208 159877 166431 158074 171268 165494 178636 176421 152176
 177234 172240 172540 181025 173052 178450 162402 159685 180087 181527
 179296 176080 161727 173752 171642 172266 174292 170482 175395 155403
 179242 162297 166387 168080 181815 163432 173994 166905 162569 172644
 166523 176349 165250 166318 164999 161249 169173 157373 174500 179870
 168953 165343 170232 172375 169640 187899 156238 158712 173582 152922
 177456 164063 173114 166861 161836 165748 168433 166477 170929 169696
 179283 169603 170278 157885 178161 172131 147106 180888 154487 171875
 15829

In [21]:
import jax.numpy as jnp
from jax import random, lax

def permutation_parent_selection(population, fitness_values, selection_method="roulette_wheel"):
  """Selects parents for a genetic algorithm using permutation representation.

  Args:
    population: A list of chromosomes in permutation representation.
    fitness_values: A list of fitness values for each chromosome in the population.
    selection_method: The parent selection method to use. Supported methods are
      "roulette_wheel" and "tournament".

  Returns:
    A tuple of two chromosomes selected as parents.
  """

  if selection_method == "roulette_wheel":
    return roulette_wheel_permutation_parent_selection(population, fitness_values)
  elif selection_method == "tournament":
    return tournament_permutation_parent_selection(population, fitness_values)
  else:
    raise ValueError("Unsupported parent selection method: {}".format(selection_method))


# def roulette_wheel_permutation_parent_selection(population, fitness_values):

#   total_fitness = np.sum(fitness_values)
#   fitness_probabilities = fitness_values / total_fitness
#   cumulative_probabilities = np.cumsum(fitness_probabilities)

#   random_value = random.random()
#   parent1_index = 0
#   for i, probability in enumerate(cumulative_probabilities):
#     if random_value < probability:
#       parent1 = population[i]
#       break

#   random_value = random.random()
#   parent2_index = 0
#   for i, probability in enumerate(cumulative_probabilities):
#     if random_value < probability:
#       parent2 = population[i]
#       break

#   return parent1, parent2


def roulette_wheel_permutation_parent_selection(population, fitness_values, key):
  key = random.PRNGKey(72)
  total_fitness = jnp.sum(fitness_values)
  fitness_probabilities = fitness_values / total_fitness
  cumulative_probabilities = jnp.cumsum(fitness_probabilities)

  def find_parent(random_value):
    def body_fun(i, parent_index):
      return lax.cond(random_value < cumulative_probabilities[i], lambda _: i, lambda _: parent_index, None)
    parent_index = lax.fori_loop(0, len(cumulative_probabilities), body_fun, 0)
    return population[parent_index]

  key1, key2 = random.split(key, 2)
  random_value1 = random.uniform(key1)
  random_value2 = random.uniform(key2)

  parent1 = find_parent(random_value1)
  parent2 = find_parent(random_value2)

  return parent1, parent2


def tournament_permutation_parent_selection(population, fitness_values, tournament_size=2):
  """Selects parents using tournament selection for permutation representation.

  Args:
    population: A list of chromosomes in permutation representation.
    fitness_values: A list of fitness values for each chromosome in the population.
    tournament_size: The size of the tournament.

  Returns:
    A tuple of two chromosomes selected as parents.
  """

  parent1 = None
  parent2 = None

  for _ in range(tournament_size):
    random_index = random.randint(0, len(population) - 1)

    if parent1 is None or fitness_values[random_index] > fitness_values[parent1]:
      parent1 = population[random_index]

    if parent2 is None or fitness_values[random_index] > fitness_values[parent2]:
      parent2 = population[random_index]

  return parent1, parent2


In [ ]:
roulette_wheel_permutation_parent_selection(population=population, fitness_values=fitness_values)

(array([24, 83, 17, 31, 60, 67, 62,  2,  9, 76, 49, 32,  0, 78, 13, 77,  1,
        44, 59, 34, 72, 92, 43, 15, 40, 46, 63, 98, 22, 94, 64, 39, 25, 70,
        16, 23, 80, 48, 93, 69, 79,  7, 68, 56, 61, 18, 75, 41, 12, 37, 55,
        33, 86, 99, 50, 90, 11, 53, 58, 87, 82, 30, 14, 89, 19, 36, 20, 85,
        29, 65, 73,  6,  8, 95, 10, 71, 51, 66,  3, 27, 21, 47, 35, 45, 81,
        96, 54, 57, 84, 91, 26, 88,  4,  5, 42, 38, 52, 28, 74, 97]),
 array([79, 68, 16, 84, 99, 61, 88, 19, 24, 48, 65, 49, 77, 82, 12,  7, 22,
        62, 95, 74, 57, 96, 97, 43, 11,  2, 41, 28,  1, 56, 75, 47, 83,  9,
        69,  6, 91, 63, 71, 30, 37, 33, 27, 85, 20, 70, 94, 13, 14, 89, 51,
        10, 80, 18, 55, 38, 52, 35, 42, 78, 81, 44, 87, 36, 86, 92, 67, 45,
        53, 25,  4, 21, 50, 40, 58, 60, 76, 15, 59, 26,  0,  8, 90, 46, 31,
        64, 73, 72, 98, 29, 17, 93, 34, 23, 32,  3,  5, 54, 39, 66]))

In [22]:
# def PMX_crossover(parent1, parent2, seed):

#     rng = np.random.default_rng(seed=seed)

#     cutoff_1, cutoff_2 = np.sort(rng.choice(np.arange(len(parent1)+1), size=2, replace=False))

#     def PMX_one_offspring(p1, p2):
#         offspring = np.zeros(len(p1), dtype=p1.dtype)
#         offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]
#         for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
#             candidate = p2[i]
#             while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
#                 candidate = p2[np.where(p1 == candidate)[0][0]]
#             offspring[i] = candidate
#         return offspring

#     offspring1 = PMX_one_offspring(parent1, parent2)
#     offspring2 = PMX_one_offspring(parent2, parent1)

#     return offspring1, offspring2

import jax.numpy as jnp
from jax import random, lax

def PMX_crossover(parent1, parent2, seed):

    key = random.PRNGKey(seed)

    cutoff_1, cutoff_2 = jnp.sort(random.choice(key, jnp.arange(len(parent1)+1), shape=(2,), replace=False))

    def PMX_one_offspring(p1, p2):
        offspring = jnp.zeros(len(p1), dtype=p1.dtype)
        offspring = offspring.at[cutoff_1:cutoff_2].set(p1[cutoff_1:cutoff_2])
        for i in jnp.concatenate([jnp.arange(0,cutoff_1), jnp.arange(cutoff_2,len(p1))]):
            candidate = p2[i]
            while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                candidate = p2[jnp.where(p1 == candidate)[0][0]]
            offspring = offspring.at[i].set(candidate)
        return offspring

    offspring1 = PMX_one_offspring(parent1, parent2)
    offspring2 = PMX_one_offspring(parent2, parent1)

    return offspring1, offspring2




# parent1 = np.random.permutation(10)
# parent2 = np.random.permutation(10)
# # Print the parents
# print("Parent 1:", parent1)
# print("Parent 2:", parent2)
# # Apply PMX and print the offspring
# offspring1, offspring2 = PMX_crossover(parent1, parent2,313)
# print("Offspring 1:", offspring1)
# print("Offspring 2:", offspring2)

In [23]:
offspring1, offspring2 = PMX_crossover(population[0], population[1],313)
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)

Offspring 1: [89 59 43 64 10 23 34  8 16 76 77 96 83 26 62 38 85 68 72  9 79 97 70 63
 42 93  0 25  2 33 95 82 92 24 13 48 50 49 98 44  7 91 37  5 47 14 53 71
 36 41 29 54 32 65 20 66 11 40 27 78 87  6 73 12 90 39 80  4 30 75 17 84
 57 18 15 67 60 74 22  3 52 61  1 88 99 51 55 45 86 56 19 31 69 21 81 46
 35 28 94 58]
Offspring 2: [46 57 86 59 47 67 97  0  3  9 29 68 60 38 25  8 22 44 15 88 70 79 39 51
 16 56 71  6 12 77 20 66  4 41 35 40 53 99 54 42 19 61 93 18 83 27 74 28
 52 30 80 21  1 84 69 36 81 75 43 82 37  7 50 23 33 10 72 92 94 87 48 73
 55 26 32 76 96 31  2 90  5 91 24 17 78 11 62 64 63 58 13 95 65 45 98 34
 85 89 49 14]


In [24]:
firstPermutation = np.random.permutation(100) # generate a random permutation of integers from 0 to 99
print(firstPermutation.shape)
secondPermutation = np.random.permutation(100) # generate another random permutation of integers from 0 to 99
PMX_crossover(firstPermutation, secondPermutation,313) # apply the PMX

(100,)


(array([36, 33, 81, 72, 45, 53, 32, 18, 43, 23, 68, 93, 62, 16, 77, 41, 94,
         5, 52, 11, 30, 12, 27, 95, 60, 39, 79, 66, 56, 48, 42,  6,  2, 92,
        73, 58, 19, 13, 38, 46, 35, 65, 44, 84, 34, 67, 89, 87, 76, 47, 55,
        96, 88, 29, 20, 14, 26, 69, 25, 97, 49, 28, 82, 91, 78, 83, 80, 17,
        70,  7, 40, 24, 31, 61, 54, 63, 51,  9, 85, 99,  4, 90, 50, 59,  0,
        21, 71,  1,  3, 15, 22, 86, 74, 10, 75, 98, 57,  8, 37, 64]),
 array([36, 96, 63, 40, 99, 74, 59, 75, 79, 37, 18, 47, 84, 23, 10,  5, 38,
        12, 94, 25, 46, 48, 17, 11, 92, 43, 87, 16, 20, 15, 51, 85, 62, 83,
         2,  8, 71, 42, 89, 68, 56,  4, 55, 91, 49, 14, 21, 32, 76, 80,  7,
        24, 82, 81, 90, 88, 26, 45, 58, 54, 22, 69, 41, 77, 52, 65, 98, 70,
        97,  1, 30, 67, 95,  0, 50,  6, 64, 44, 19, 60, 34, 61, 57, 13, 39,
        78,  9, 33, 28, 66, 73, 86, 27, 72, 93, 35, 53,  3, 31, 29]))

In [25]:
# def cxPartialyMatched(ind1, ind2):
#     size = min(len(ind1), len(ind2))
#     p1, p2 = [0] * size, [0] * size

#     for i in range(size):
#         p1[ind1[i]] = i
#         p2[ind2[i]] = i
#     cxpoint1 = random.randint(0, size)
#     cxpoint2 = random.randint(0, size - 1)
#     if cxpoint2 >= cxpoint1:
#         cxpoint2 += 1
#     else:  # Swap the two cx points
#         cxpoint1, cxpoint2 = cxpoint2, cxpoint1

#     for i in range(cxpoint1, cxpoint2):
#         temp1 = ind1[i]
#         temp2 = ind2[i]
#         ind1[i], ind1[p1[temp2]] = temp2, temp1
#         ind2[i], ind2[p2[temp1]] = temp1, temp2
#         p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
#         p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

#     return ind1, ind2
import jax.numpy as jnp
from jax import random, lax

def cxPartialyMatched(ind1, ind2, key):

    size = min(len(ind1), len(ind2))
    p1, p2 = [0] * size, [0] * size

    for i in range(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i

    key1, key2 = random.split(key, 2)
    cxpoint1 = random.randint(key1, 0, size)
    cxpoint2 = random.randint(key2, 0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    for i in range(cxpoint1, cxpoint2):
        temp1 = ind1[i]
        temp2 = ind2[i]
        ind1 = ind1.at[i].set(temp2).at[p1[temp2]].set(temp1)
        ind2 = ind2.at[i].set(temp1).at[p2[temp1]].set(temp2)
        p1 = jnp.array(p1).at[temp1].set(p1[temp2]).at[temp2].set(p1[temp1])
        p2 = jnp.array(p2).at[temp1].set(p2[temp2]).at[temp2].set(p2[temp1])

    return ind1, ind2


offspring1, offspring2 = cxPartialyMatched(population[0], population[1])
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)

Offspring 1: [97 55 51 59 60  9 70 28 80 88 29 42 68  0 19 26 12  3 32 48 39 34 10 11
 44 58 71  7 23 33 20 78 92 94 22 37 73 36 64 96  6 91 56 38 83 27 74 89
 82 30 15 21 24 50 65 52 98 75 43  5 93 13 31  2 90 79 77  4 41 40 17 84
 25 18 72 67 47 69 85 16 66 61  1 76 99 63 57 45 86 87 62 53 95 54 81 46
 35  8 49 14]
Offspring 2: [ 8 59 27 45 79  2 46 26 44 67 33 47 83 18 57  0 35 42 77 88 34 70 39 86
 96 93 28 19 85 90 53 52  4 30 62 75 74 49 21  3  7 61 40 66 60 43 84 71
 99  1 29 81 72 95 20 82 63 17 14 78 56  6 73 23 16 10 15 92 94 37 76 50
 55 38 32  9 68 31 12 80  5 91 24 48 36 11 25 54 51 58 13 69 65 64 98 97
 22 89 41 87]


In [ ]:
def erx_crossover(parent1, parent2):
    """Performs Edge Recombination Crossover (ERX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    offspring = []
    visited = set()

    # Start from a random gene in parent1
    gene = random.choice(parent1)
    offspring.append(gene)
    visited.add(gene)

    while len(offspring) < len(parent1):
        # Find the next gene to add
        next_gene = None

        # Check if the next gene in parent2 is not in the visited set
        next_gene_in_parent2 = parent2[(parent1.index(gene) + 1) % len(parent1)]
        if next_gene_in_parent2 not in visited:
            next_gene = next_gene_in_parent2

        # Check if the next gene in parent1 is not in the visited set
        if next_gene is None:
            next_gene_in_parent1 = parent1[(parent2.index(gene) + 1) % len(parent2)]
            if next_gene_in_parent1 not in visited:
                next_gene = next_gene_in_parent1

        # If no valid next gene found, break
        if next_gene is None:
            break

        # Add the next gene to the offspring and mark it as visited
        offspring.append(next_gene)
        visited.add(next_gene)

        # Update the current gene
        gene = next_gene

    return offspring

In [ ]:
def ox_crossover(parent1, parent2):
    """Performs Order Crossover (OX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    crossover_points = random.sample(range(1, len(parent1)), 2)
    slice_start, slice_end = crossover_points[0], crossover_points[1]

    slice1 = parent1[slice_start:slice_end]
    remaining_genes1 = set(parent1) - set(slice1)

    mapping = {}
    for i in range(slice_start, slice_end):
        gene1 = parent1[i]
        gene2 = parent2[i]

        if gene1 in remaining_genes1:
            mapping[gene1] = gene2
            remaining_genes1.remove(gene1)

    offspring = []
    for gene in parent2:
        if gene in mapping:
            offspring.append(mapping[gene])
        elif gene not in slice1:
            offspring.append(gene)

    for gene in slice1:
        offspring.append(gene)

    return offspring

In [ ]:
def cycle_crossover(parent1, parent2):
    """
    Performs Cycle Crossover (CX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    cycle_start = random.randrange(len(parent1))
    cycle_genes1 = set(parent1[cycle_start:])
    cycle_genes2 = set(parent2[cycle_start:])

    offspring = list(parent1[:cycle_start])
    for gene in parent2:
        if gene in cycle_genes1:
            offspring.append(gene)
            cycle_genes1.remove(gene)

            if not cycle_genes1:
                break
        else:
            cycle_genes2.add(gene)

    for gene in cycle_genes2:
        offspring.append(gene)

    return offspring

In [26]:
def swap_mutation(chromosome):
    """Performs Swap mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    i, j = random.sample(range(len(chromosome)), 2)
    chromosome[i], chromosome[j] = chromosome[j], chromosome[i]
    return chromosome

In [27]:
def scramble_mutation(chromosome):
    """Performs Scramble mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    mutation_length = random.randrange(2, len(chromosome))
    start_index = random.randrange(len(chromosome) - mutation_length)

    sub_chromosome = chromosome[start_index:start_index + mutation_length]
    random.shuffle(sub_chromosome)
    chromosome[start_index:start_index + mutation_length] = sub_chromosome

    return chromosome

In [28]:
def insert_mutation(chromosome):
    """Performs Insert mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    i = random.randrange(len(chromosome))
    j = random.randrange(len(chromosome))

    if i != j:
        gene = chromosome.pop(i)
        chromosome.insert(j, gene)

    return chromosome

In [29]:
def inversion_mutation(chromosome):
    """Performs Inversion mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    mutation_length = random.randrange(2, len(chromosome))
    start_index = random.randrange(len(chromosome) - mutation_length)

    sub_chromosome = chromosome[start_index:start_index + mutation_length]
    sub_chromosome.reverse()
    chromosome[start_index:start_index + mutation_length] = sub_chromosome

    return chromosome

In [30]:
# import numpy as np

def inversion_mutation(permutation):
  # randomly select two positions in the permutation
  i, j = np.random.choice(len(permutation), size=2, replace=False)
  # make sure i is smaller than j
  if i > j:
    i, j = j, i
  # invert the segment between i and j
  permutation[i:j+1] = permutation[i:j+1][::-1]
  return permutation


In [31]:
# import numpy as np

# def inversion_mutation(chromosome):

#   # Select two random indices in the chromosome.
#   index1 = np.random.randint(0, len(chromosome))
#   index2 = np.random.randint(0, len(chromosome))

#   while index1 == index2:
#     index2 = np.random.randint(0, len(chromosome))

#   chromosome[index1:index2 + 1] = chromosome[index1:index2 + 1][::-1]

#   return chromosome
import jax.numpy as jnp
from jax import random, lax

def inversion_mutation(chromosome, key):

  # Select two random indices in the chromosome.
  key1, key2 = random.split(key, 2)
  index1 = random.randint(key1, (1,), 0, len(chromosome))
  index2 = random.randint(key2, (1,), 0, len(chromosome))

  while index1 == index2:
    key2, _ = random.split(key2, 2)
    index2 = random.randint(key2, (1,), 0, len(chromosome))

  chromosome = chromosome.at[index1:index2 + 1].set(chromosome[index1:index2 + 1][::-1])

  return chromosome

# Fake data
chromosome = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Mutate the chromosome
mutated_chromosome = inversion_mutation(chromosome)

# Print the mutated chromosome
print(mutated_chromosome)


[ 1  2  3  4  5  6  7  8  9 10]


In [ ]:
# import unittest
# import numpy as np

# def inversion_mutation(permutation):
#     print('permutation is : ')
#     print(permutation)
#     # randomly select two positions in the permutation
#     i, j = np.random.choice(len(permutation), size=2, replace=False)
#     # make sure i is smaller than j
#     if i > j:
#         i, j = j, i
#     # invert the segment between i and j
#     permutation[i:j+1] = permutation[i:j+1][::-1]
#     return permutation

# class TestInversionMutation(unittest.TestCase):
#     def test_inversion_mutation(self):
#         # Test case 1: permutation with length 5
#         permutation = [1, 2, 3, 4, 5]
#         mutated_permutation = inversion_mutation(permutation)
#         self.assertEqual(len(mutated_permutation), len(permutation))
#         self.assertNotEqual(mutated_permutation, permutation)

#         # Test case 2: permutation with length 10
#         permutation = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
#         mutated_permutation = inversion_mutation(permutation)
#         self.assertEqual(len(mutated_permutation), len(permutation))
#         self.assertNotEqual(mutated_permutation, permutation)

# if __name__ == '__main__':
#     unittest.main()


In [32]:
PMX_crossover(population[0], population[1],313)

(array([ 8, 59, 14, 45, 79,  2, 46, 26, 44, 67, 33, 47, 60, 18, 57,  0, 35,
        42, 77, 88, 34, 70, 39, 86,  3, 40, 28, 19, 85, 90, 53, 78, 92, 94,
        22, 37, 73, 36, 64, 96,  6, 91, 56, 38, 83, 27, 74, 89, 82, 30, 15,
        21, 24, 50, 65, 52, 98, 75, 43,  5, 93, 13, 31, 23, 16, 10, 29,  4,
         1, 17, 76, 95, 55, 66, 32,  9, 68, 84, 12, 80, 99, 61, 72, 48, 49,
        11, 25, 54, 51, 58,  7, 69, 20, 81, 63, 97, 62, 71, 41, 87]),
 array([97, 55, 51, 59, 83,  9, 70, 28, 80, 88, 15, 42, 68,  0, 19, 26, 12,
        96, 32, 48, 39, 34, 10, 11, 44, 58, 89, 13, 23, 33, 65, 52,  4, 30,
        62, 75, 74, 49, 21,  3,  7, 61, 40, 66, 60, 43, 84, 71, 99,  1, 29,
        81, 72, 95, 20, 82, 63, 17, 14, 78, 56,  6, 73,  2, 90, 79, 77, 92,
        41, 93, 37, 31, 25, 18, 24, 67, 47, 69, 85, 16, 38, 91, 94, 76,  5,
        98, 57, 45, 86, 87, 22, 53, 50, 54, 64, 46, 35,  8, 36, 27]))

In [3]:
# def roulette_wheel_selection(population, fitness_values):
#     total_fitness = np.sum(fitness_values)
#     fitness_probabilities = fitness_values / total_fitness
#     cumulative_probabilities = np.cumsum(fitness_probabilities)

#     random_index = random.random()
#     for i, probability in enumerate(cumulative_probabilities):
#         # print(f'i is {i}')
#         if random_index < probability:
#             return population[i]
# def roulette_wheel_selection(population, fitness_values):
#     total_fitness = np.sum(fitness_values)
#     fitness_probabilities = fitness_values / total_fitness
#     cumulative_probabilities = np.cumsum(fitness_probabilities)

#     random_value = random.random()
#     for i, probability in enumerate(cumulative_probabilities):
#         if random_value < probability:
#             selected_chromosome = population[i]
#             break

#     # Shuffle the selected chromosome to ensure no bias in selecting the next parent
#     # np.random.shuffle(selected_chromosome)

#     return selected_chromosome

# def tournament_selection(population , cities = city_data, tournament_size = 5):
#     # Create a tournament
#     tournament = random.sample(population, tournament_size)
#     # Select the best individual among the tournament
#     best_individual = min(tournament, key=lambda x: calculate_distance(cities, x))
#     return best_individual

def genetic_algorithm(population_size=500, crossover_rate=0.7, mutation_rate=0.1, max_generations=700,cities = city_data):
    chromosomes = generate_random_chromosomes(500)
    print(len(chromosomes))
    fitness_values = []
    for chromosome in chromosomes :
        fitness_value = calculate_fitness_np(chromosome, cities)
        fitness_values.append(fitness_value)

    generation_without_improvement = 0

    for generation in range(max_generations):
        # Select parents using roulette wheel selection
        parents = []
        for i in range(population_size):
            parent1 , parent2 = permutation_parent_selection(chromosomes, fitness_values)
            parents.append((parent1, parent2))

        offsprings = []
        for parent1, parent2 in parents:
            # print(parent1)
            # break
            if random.random() < crossover_rate:
                offspring1 ,offspring2 = PMX_crossover(parent1, parent2,153)
            else:
                offspring1 = parent1
                offspring2 = parent2

            offsprings.append(offspring1)
            offsprings.append(offspring2)

        # Apply mutation
        for offspring in offsprings:
            if random.random() < mutation_rate:
                # print('offspring is' )
                # print(type(offspring) )
                offspring = inversion_mutation(offspring)

        # Evaluate offspring fitness
        offspring_fitness_values = []
        for offspring in offsprings :
            # calculate_fitness(offspring, cities)
            offspring_fitness_value = calculate_fitness_np(offspring)
            if offspring_fitness_value < 21285 :
                print(offspring_fitness_value)
            offspring_fitness_values.append(offspring_fitness_value)

        # Combine parents and offspring to form a new population
        new_population = np.empty_like(chromosomes)
        new_fitness_values = np.empty_like(fitness_values)

        for i in range(population_size):
            if offspring_fitness_values[i] < fitness_values[i]:
                new_population[i] = offsprings[i]
                new_fitness_values[i] = offspring_fitness_values[i]
            else:
                new_population[i] = chromosomes[i]
                new_fitness_values[i] = fitness_values[i]

        chromosomes = new_population
        fitness_values = new_fitness_values

        # if np.array_equal(fitness_values, offspring_fitness_values):
        if jnp.array_equal(fitness_values, offspring_fitness_values):
        # if fitness_values < offspring_fitness_values : 
            generation_without_improvement += 1

            if generation_without_improvement >= 20:
                break
        else:
            generation_without_improvement = 0

    best_chromosome = chromosomes[fitness_values.argmax()]
    best_fitness = np.min(fitness_values)

    return best_chromosome, best_fitness


NameError: name 'city_data' is not defined

In [ ]:
result = genetic_algorithm()

500


In [ ]:
print(result)

(array([79, 22, 94, 16, 88, 70, 26, 66, 65,  4, 97, 56, 75, 19, 90, 84, 10,
       72, 18, 24, 38, 99, 36, 74, 15, 11, 17, 32, 45, 47, 58, 28, 61,  7,
        9, 57, 55, 83, 12, 20, 62, 86, 35, 27, 34, 43, 29,  3, 46, 14, 71,
       41,  0, 48, 52, 30, 85, 39, 96, 78, 33, 76, 13,  5, 37, 82, 95, 54,
       40, 87, 51, 81, 25, 68, 73, 50, 44,  2, 64, 69, 67, 93, 23, 98, 77,
       60, 91, 59, 21,  6, 92,  8, 80, 31, 89, 42,  1, 63, 49, 53]), 32247)
